In [1]:
# Install dependencies

import sys
!{sys.executable} -m pip install hdfs

In [2]:
pip install delta-spark

Note: you may need to restart the kernel to use updated packages.


In [9]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/TrabalhoPL/gold'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
# create gold database
spark.sql(
    """
    DROP DATABASE IF EXISTS Projeto_gold CASCADE
    """
)
spark.sql(
    """
    create database Projeto_gold location 'hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db'
    """
)


DataFrame[]

In [10]:
from pyspark.sql.functions import substring, avg, sum

# read air_quality from the silver tables
hdfs_path = "hdfs://hdfs-nn:9000/TrabalhoPL/silver/Projeto.db/Tabela_Agua"

agua = spark\
             .read\
             .load(hdfs_path)

agua.show()

+---------+-----------+-----------+-----------+--------------------+------------+------------+------------+-------------+----+
|Sample_Id|Sample_Site|Sample_Date|Sample_Time|             Analyte|      Status|Final_Result|       Units| Stream_Group| Ano|
+---------+-----------+-----------+-----------+--------------------+------------+------------+------------+-------------+----+
| C-175259|       E16I| 1997-01-02|       8:32|Solids, Total Sus...|Desconhecido|           3|        mg/L|       Esopus|1997|
| C-175259|       E16I| 1997-01-02|       8:32|                  pH|Desconhecido|        6.73|Desconhecida|       Esopus|1997|
| C-175259|       E16I| 1997-01-02|       8:32|         Temperature|Desconhecido|           1|           C|       Esopus|1997|
| C-175259|       E16I| 1997-01-02|       8:32|      Apparent Color|Desconhecido|           6|          CU|       Esopus|1997|
| C-175259|       E16I| 1997-01-02|       8:32|     Scent Character|Desconhecido|           0|Desconhecida|    

In [11]:
from pyspark.sql.functions import count
gold_agua = agua \
    .groupBy("Ano", "Status")\
    .agg(
        count(agua.Status).alias("Contagem")
    ) \
   
gold_agua.toPandas()


,Ano,Status,Contagem
0,1999,Desconhecido,85759
1,1997,Desconhecido,85318
2,1998,Desconhecido,85541
3,2001,Desconhecido,83135
4,1996,Desconhecido,67851
...,...,...,...
66,2013,Cancel,1159
67,2013,Failure,439
68,2020,Failure,278
69,2020,Cancel,887


In [13]:
# create air_quality table in gold
spark.sql(
    """
    DROP TABLE IF EXISTS Projeto_gold.Tabela_Agua
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE Projeto_gold.Tabela_Agua (
        Ano INT,
        Status VARCHAR(500), 
        Contagem LONG
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db/Tabela_Agua/'
    """
)
        

DataFrame[]

In [14]:
# write to delta table
gold_agua \
    .write  \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db/Tabela_Agua/")

In [15]:
# check the results in the table
spark.table("Projeto_gold.Tabela_Agua").toPandas()


,Ano,Status,Contagem
0,1999,Desconhecido,85759
1,1997,Desconhecido,85318
2,1998,Desconhecido,85541
3,2001,Desconhecido,83135
4,1996,Desconhecido,67851
...,...,...,...
66,2013,Cancel,1159
67,2013,Failure,439
68,2020,Failure,278
69,2020,Cancel,887


In [16]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db/Tabela_Agua/`
""").show()

++
||
++
++



In [17]:
spark.sql("""
DROP TABLE IF EXISTS Projeto_gold.Tabela_Agua_Presto 
""").show()

spark.sql("""
    CREATE EXTERNAL TABLE Projeto_gold.Tabela_Agua_Presto (
        Ano INT,
        Status VARCHAR(500), 
        Contagem LONG
    )
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db/Tabela_Agua/_symlink_format_manifest/'
""").show()

++
||
++
++

++
||
++
++

